# Análisis exploratorio de *items_ordered_2years*

En este cuadeerno se detalla el proceso de análisis de datos de los pedidos realizados

# Índice de Contenidos

1. Importación de paquetes
2. Carga de datos
3. Análisis de los datos

    3.1. Variables

    3.2. Duplicados

    3.3. Clientes

    3.4. Ventas totales

    3.5. Ventas por localización
    
4. Conclusiones

## Importación de paquetes

Cargamos las librerías a usar.

In [53]:
import pandas as pd
import plotly.express as px
import random

seed = 124
random.seed(seed)

## Carga de datos

Abrimos el fichero y lo cargamos para poder manejar la información

In [54]:
df_items = pd.read_csv('../Data/items_ordered_2years.txt', sep='|', on_bad_lines='skip',parse_dates=['created_at'])
df_items.sample(5, random_state=seed)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
117430,3144f70e73aabd87886a92c499134ec6,84492c3f9eeefe8f358b9f78d42e6d51,2017-06-06 19:47:26,15054,1,37.8377,41.28,9.0,cc2905f9c3e5fcda4d1878b57c486a54,VITORIA,01007
835866,8dbfe1ffec173f2e8a44a85dd66278ab,64f6ad39241754101366880979e9b831,2018-11-24 14:30:23,55454,1,7.0833,8.85,20.0,f74d36ac2995966d36a8c7c29c0167fa,Zaragoza,50019
298248,a4403564192ddabe89734bcfddc32eb2,060541582b058837fe2708225b7a10d9,2017-11-27 09:23:45,2284,1,6.8250,8.05,8.0,f8ae06846397f55bc9e7c063f960207e,CIUDAD RODRIGO,37500
103857,d52995ca3b208edbffe643d5ad2e5b50,f033b7a59a6dbe537891c73b20b30a83,2017-05-13 08:27:06,12051,1,2.6111,4.10,5.0,121aea0f7e1978bf3f457d50d20aaa03,POZOBLANCO,14400
180683,e753adf54a219db2e06fe9345ae4eb9a,587ab92d5cb201abeb9b0506fea500d5,2017-07-29 18:32:54,11242,1,4.7902,7.75,9.0,facafe93eba27034473c7b57d8902e0d,CIUDAD REAL,13002


## Análisis de los datos

En este apartado se analizarán todas las características de las variables

In [55]:
print("Dimensiones de los datos:", df_items.shape)

Dimensiones de los datos: (930905, 11)


In [56]:
df_items.isna().sum()

num_order              0
item_id                0
created_at             0
product_id             0
qty_ordered            0
base_cost           2402
price                  0
discount_percent       0
customer_id            0
city                3007
zipcode             3004
dtype: int64

Existen datos perdidos o no proporcionados de 3 variables distintas

In [57]:
percent_missing = df_items.isnull().sum() * 100 / df_items.shape[0]
df_missing_values = pd.DataFrame({ 
    'column_name': df_items.columns,
    'percent_missing': percent_missing
})
df_missing_values.sort_values(by='percent_missing',ascending=False,inplace=True)
print(df_missing_values.to_string(index=False))

     column_name  percent_missing
            city         0.323019
         zipcode         0.322697
       base_cost         0.258028
       num_order         0.000000
         item_id         0.000000
      created_at         0.000000
      product_id         0.000000
     qty_ordered         0.000000
           price         0.000000
discount_percent         0.000000
     customer_id         0.000000


Aunque haya datos perdidos, estos no representan una gran proporción respecto al dataset

In [58]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930905 entries, 0 to 930904
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   num_order         930905 non-null  object        
 1   item_id           930905 non-null  object        
 2   created_at        930905 non-null  datetime64[ns]
 3   product_id        930905 non-null  int64         
 4   qty_ordered       930905 non-null  int64         
 5   base_cost         928503 non-null  float64       
 6   price             930905 non-null  float64       
 7   discount_percent  930905 non-null  float64       
 8   customer_id       930905 non-null  object        
 9   city              927898 non-null  object        
 10  zipcode           927901 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 60.4+ MB


Podemos observar que tenemos 1 variable de tipo fecha, 2 variables que son números enteros (el identificador de producto y la cantidad pedida), 3 variables son números decimales (precio base, precio de venta y porcentaje de descuento) y las demás variables que son clásificadas como tipo **object** también las podemos considerar de tipo *string*

In [59]:
df_items.describe()

,product_id,qty_ordered,base_cost,price,discount_percent
count,930905.000000,930905.000000,928503.00000,930905.000000,930905.000000
mean,28608.778755,1.354653,8.49795,10.883495,8.188974
std,26808.254585,1.280747,77.50866,9.888217,6.760173
min,85.000000,1.000000,-164.79740,0.000000,1.000000
25%,6480.000000,1.000000,3.30470,4.690000,5.000000
50%,14765.000000,1.000000,6.09540,8.220000,6.000000
75%,50567.000000,1.000000,10.46500,13.960000,10.000000
max,98205.000000,120.000000,47251.00000,650.830000,100.000000


Al echar un vistazo por encima nos podemos dar cuenta de un par de cosas un tanto *extrañas*:
* El mínimo precio base es negativo
* El mínimo precio de venta es 0
* El mínimo porcentaje de descuento es 1, por lo tanto no hay ningún pedido que no tenga aplicado un porcentaje de descuento

In [60]:
df_items.describe(include=object)

,num_order,item_id,customer_id,city,zipcode
count,930905,930905,930905,927898,927901
unique,278040,906266,118796,20993,11116
top,8edd335c2683fd27bdb93326db66b99f,642b9b87df5b13e91ce86962684c2613,60d097f001dd2ee8c9989434307a5c2b,Madrid,28050
freq,60,3,870,69308,5355


Vemos que el número de pedido se repite y esto se debe a que por cada producto distinto que existen dentro de un pedido necesitamos un registro nuevo, y por eso se repide el identificador de pedido

### Variables

Ahora realizaremos un análisis más específico de cada variable

#### *num_order*

Esta variable es el identificador de los pedidos y la podemos ver repetida ya que en un mismo pedido puede haber diferentes productos, y para cada producto distinto dentro de un mismo pedido se usan distintas entradas.

In [61]:
orders = df_items['num_order'].nunique()
print(f'Hay {orders} pedidos registrados')

Hay 278040 pedidos registrados


In [62]:
df_items['num_order'].value_counts()

8edd335c2683fd27bdb93326db66b99f    60
32bc727524a84262812580f0cae3b0d8    57
94f3ba50d99564135beac0041365c6c9    56
63e18c8dc7557fc2a2794c9b44c0a0c2    55
826d04fc123bae848bd5704bd517ce1a    55
                                    ..
d33eaff239b4bc93d4a2df5ac496daae     1
af61815e3df9330dceb0f36a60e25487     1
346766a5c8d3725d988c64ff46ca6b85     1
fb34477f573ffa3cb117b4ddfe68f5a7     1
881455cb3a7483be5a802af92f2256c8     1
Name: num_order, Length: 278040, dtype: int64

El máximo de líneas de pedidos son 60, mientras que hay pedidos con tan solo una línea. 

#### *item_id*

La variable *item_id* se usa para identificar el producto dentro del pedido, por lo tanto un mismo producto que este en pedidos distintos tendrá distintos *item_id*

In [63]:
items = df_items['item_id'].nunique()
print(f'Identificadores : {items}')
print(f'Registros : {df_items.shape[0]}')

Identificadores : 906266
Registros : 930905


Como se ve no tenemos identificadores de items como registros del dataset lo que nos lleva a pensar que podría haber registros repetidos

In [64]:
df_items['item_id'].value_counts()

642b9b87df5b13e91ce86962684c2613    3
b65a8a92ca1c759728dab9f559ca110f    2
2923056a8d1e6ed45515fa81abb28cd2    2
9637187d92b5f65bdf2274192cc383b9    2
147b2733c3c7ede56fc0216dbecaa086    2
                                   ..
f9889940349661aabca5f39a2f48c0b4    1
51dfa451f3dce80988c7513d993f8679    1
0ee2102c149c5e12ea625e96570418c4    1
163022e8334c7399148532a3bf705e5e    1
fd0c1bb9a90dce3f3b2099990f019262    1
Name: item_id, Length: 906266, dtype: int64

In [65]:
df_items[df_items['item_id'] == '642b9b87df5b13e91ce86962684c2613']

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
874752,5672b1fa40b5398a40a1790374ebffa2,642b9b87df5b13e91ce86962684c2613,2018-12-17 01:19:35,7391,2,6.4084,9.76,57.0,d8d4d4aadb6c893d715015a8a2a04f94,Valladolid,47012
874753,5672b1fa40b5398a40a1790374ebffa2,642b9b87df5b13e91ce86962684c2613,2018-12-17 01:19:35,7391,2,6.4084,9.76,57.0,d8d4d4aadb6c893d715015a8a2a04f94,Valladolid,47012
874754,5672b1fa40b5398a40a1790374ebffa2,642b9b87df5b13e91ce86962684c2613,2018-12-17 01:19:35,7391,2,6.4084,9.76,57.0,d8d4d4aadb6c893d715015a8a2a04f94,Valladolid,47012


Nuestras sospechas se confirman, ya que vemos que el mismo pedido tiene las tres mismas líneas. Por tanto, se trata de datos duplicados

#### *created_at*

Instante en el que se realizó el pedido

In [66]:
df_items['created_at'].sample(1,random_state=seed)

117430   2017-06-06 19:47:26
Name: created_at, dtype: datetime64[ns]

El formato de la fecha y tiempo es *YYYY-MM-DD HH:MM:SS*

#### *product_id*

Identificador del producto

In [67]:
products = df_items['product_id'].nunique()
print(f'Hay {products} identificadores de productos')

Hay 26396 identificadores de productos


In [68]:
# Productos en más frecuentas en los pedidos
df_items['product_id'].value_counts()

13109    2849
7349     2759
7348     2571
43862    2544
10516    2226
         ... 
7397        1
87691       1
9778        1
84466       1
97587       1
Name: product_id, Length: 26396, dtype: int64

Los productos más comunes en los pedidos, no son exactamente los mismos que los que más cantidad se piden.

In [69]:
# Productos más vendidos (cantidad)
df_best_products = df_items.groupby("product_id",as_index=False).agg({"num_order":"count", "qty_ordered":"sum"}).sort_values("qty_ordered", ascending=False)
df_best_products.head()

,product_id,num_order,qty_ordered
3694,7348,2571,10773
3695,7349,2759,9461
3892,7629,898,8810
3693,7347,1527,7188
13586,43862,2544,6541


#### *qty_ordered*

Cantidad pedida de cierto producto para un cierto pedido

In [70]:
df_q = df_items.groupby('num_order',as_index=False).agg(total_qty=('qty_ordered','sum'),n_items=('item_id','count')).sort_values('total_qty',ascending=False)
df_q.head(10)

,num_order,total_qty,n_items
155083,8edd335c2683fd27bdb93326db66b99f,205,60
140797,8190f04869bb51f919c5fb6c31c38d61,175,8
84949,4e52e54a3f585cb3ea99cda90edf5efb,170,2
197281,b581322c8b9ff5a742083b21fdea111b,120,15
236782,da19eb62bff1d06538a37ff5990e050c,120,1
4833,047cb62fdb2a456d282465fde9175f43,115,6
102339,5e4bc005dcfdfa48fea14fa6a04fe879,111,8
90264,5333f97d62651050ee56e22d6c4e0515,111,8
25480,178896521a4b6aac03f220c00fc5d3e1,110,26
180400,a5e994e10285c621eb175dfb4b86af42,105,2


In [71]:
df_q.tail(10)

,num_order,total_qty,n_items
147868,881f259eb3b8ea82612ea15554e40941,1,1
232110,d5c6bd0b5ba4540f54dfdde68f0ab01b,1,1
147858,881cf5c9a5839af996a170b3c0b15e9a,1,1
46384,2ad1d53c6df75ac61a266e51f69ffb3f,1,1
232115,d5c71c9bd0e38ad83dcaa7b910391860,1,1
232116,d5c721b1b29c8dedc104c13a9eb726bd,1,1
232117,d5c740b9c8caee5fd558bccca1e357ca,1,1
232118,d5c8105d63704447436d6bf134f9248a,1,1
147846,881af40fe9368a9a5826a3f4ebdeae69,1,1
175604,a17376da967af87bdc36e084ca45f3c1,1,1


En el resumen de la descripción del conjunto de datos, ya veíamos que los mínimos y máximos de las cantidades que se piden eran bastante dispares. Con esta tabla vemos como se reparten las cantidades pedidas respecto a los productos dentro del pedido.

In [72]:
df_q['total_qty'].value_counts().head(10)

2     49574
1     47440
3     39832
4     35624
5     26570
6     22356
7     14185
8     11620
9      7362
10     6318
Name: total_qty, dtype: int64

In [73]:
df_q[df_q['total_qty'] > 10].shape[0]

17159

Vemos que lo más común es que en los pedidos haya entre *2* y *10* unidades de productos. Aun así hay un número sustancia del pedidos que tienen cantidades superiores a las *10* unidades.

#### *base_cost*

Precio de coste de una unidad de producto sin IVA

Como vimos al principio esta columna tiene valores perdidos. Veamos si sería posible recuperarlos mirando *base_cost* del mismo producto en otros pedidos.

In [74]:
products = df_items[df_items['base_cost'].isna()]['product_id'].drop_duplicates().to_list()
print(f'Nº productos con precios nulos: {len(products)}')
df_pr = df_items[(df_items['product_id'].isin(products)) & (df_items['base_cost'].notna())][['product_id','base_cost']]
print(f"Nº productos que se puede recuperar base_cost: {df_pr['product_id'].nunique()}")
df_pr.groupby('product_id',as_index=False).agg(n_cost=('base_cost','count'),mean_cost=('base_cost','mean')).sort_values('n_cost',ascending=False)

Nº productos con precios nulos: 564
Nº productos que se puede recuperar base_cost: 499


,product_id,n_cost,mean_cost
351,15484,1895,0.000000
375,16899,639,0.813194
146,9387,609,12.674422
96,7499,578,4.546056
221,11239,338,3.822018
...,...,...,...
119,8426,1,3.929200
474,24319,1,5.920000
107,7749,1,3.652800
309,13307,1,2.169600


Parece ser que se puede recuperar bastantes valores, eso sí hay que decidir que valores imputar porque los productos varían de precio en los diferentes pedidos. Lo más extraño a mencionar es que el precio de coste medio de uno de los productos sea directamente un cero.

In [75]:
neg_bcost = df_items[df_items['base_cost'] < 0].shape[0]
print(f'Registros con base_cost negativo: {neg_bcost}')

Registros con base_cost negativo: 46


#### *price*

Precio unitario de los productos.

In [76]:
pr_zero = df_items[df_items['price'] == 0].shape[0]
null_benefits = df_items[(df_items['price'] < df_items['base_cost'])][['product_id','base_cost','price','discount_percent']]
print(f'Productos con precio cero: {pr_zero}')
print(f'Líneas pedido sin beneficios: {null_benefits.shape[0]}')

Productos con precio cero: 61
Líneas pedido sin beneficios: 10729


Existen registros con precios nulos y también bastantes líneas de pedidos en los que nos se generan beneficios. Puede que estas últimas se traten de descuentos aplicados.

In [77]:
null_benefits.corr()

,product_id,base_cost,price,discount_percent
product_id,1.000000,0.037836,0.098569,0.010467
base_cost,0.037836,1.000000,0.060518,-0.014418
price,0.098569,0.060518,1.000000,0.033838
discount_percent,0.010467,-0.014418,0.033838,1.000000


Donde nos surge la gran duda es en que no hay correspondencia entre los costes, precios y descuentos de los productos. Por tanto, podemos intuir que los precios sobre los que se aplican los descuentos no son estos, sino otros, ya que nosotros tenemos los precios finales.

#### *discount_percent*

Porcentaje de descuento aplicado, pero no al precio con el que contamos nosotros.

In [78]:
print(f"Líneas con descuento: {df_items[df_items['discount_percent'] > 0].shape[0]}")
print(f'Lineas total: {df_items.shape[0]}')

Líneas con descuento: 930905
Lineas total: 930905


Algo que llama la atención es que todos las líneas de pedidos tienen descuentos

In [79]:
print(f"Cantidad de porc. desc.: {df_items['discount_percent'].nunique()}")
df_items['discount_percent'].value_counts().sort_values(ascending=False).head(10)

Cantidad de porc. desc.: 45


5.0     420479
10.0    158901
7.0     100545
8.0      60444
6.0      54337
20.0     50545
9.0      35230
15.0     29645
11.0     10079
25.0      2560
Name: discount_percent, dtype: int64

In [80]:
df_items['discount_percent'].value_counts().sort_values(ascending=False).tail(10)

58.0000    17
38.0000    16
29.0000    15
1.0000     15
12.5631    13
37.0000    12
33.0000     5
32.0000     4
65.0000     3
34.0000     3
Name: discount_percent, dtype: int64

Los porcentajes de descuento más comunes son por debajo de *25%*, mientras que los menos comunes oscilan entre el *30% y 50%*.

#### *customer_id*

El identificador del cliente que ha realizado el pedido

In [81]:
clients = df_items['customer_id'].nunique()
print(f'{clients} clientes han realizado pedidos')

118796 clientes han realizado pedidos


In [82]:
df_items['customer_id'].sample(5,random_state=seed)

117430    cc2905f9c3e5fcda4d1878b57c486a54
835866    f74d36ac2995966d36a8c7c29c0167fa
298248    f8ae06846397f55bc9e7c063f960207e
103857    121aea0f7e1978bf3f457d50d20aaa03
180683    facafe93eba27034473c7b57d8902e0d
Name: customer_id, dtype: object

Este campo es un *hash* con el fin de anonimizar los datos personales.

#### *city* y *zipcode*

Estas variables indican la ciudad en la cuál se han realizado el pedido

In [83]:
print(f"Nº de ciudades: {df_items['city'].nunique()}")
print(f"Nº de ciudades: {df_items['zipcode'].nunique()}")

Nº de ciudades: 20993
Nº de ciudades: 11116


Vimos antes que estas dos columnas tienen un número similar de registros nulos, lo que da a entender que puede que coincidan.

In [84]:
df_items[(df_items.city.isna()) & (df_items.zipcode.isna())].shape[0]

2910

Tenemos 2910 registros donde no tenemos ninguna información de la dirección, ni la ciudad ni  el código postal. A pesar de que no son demasiados registros, es información que no podremos recuperar de ninguna manera, ya que los datos están anonimizados

In [85]:
cities = df_items[(df_items['city'].notna()) & (df_items['city'].str.contains('alba'))]['city']
cities.value_counts()

Collado Villalba          620
albacete                  470
Collado villalba          117
collado villalba           91
Villalba de la Sierra      82
                         ... 
Collaso villalba            1
La Puebla de montalban      1
argamasilla de alba         1
villalba del rey            1
Montalbanejo                1
Name: city, Length: 62, dtype: int64

Haciendo una búsqueda sencilla de cadenas, vemos que el campo no está normalizado y hay nombres de ciudades escritos de diferentes maneras. 

In [86]:
# Normalizamos los nombres de ciudades
indexes = df_items['city'].notna().index
df_items.loc[indexes, 'city'] = df_items.loc[indexes, 'city'].apply(lambda x: str(x).lower().strip())
print(f"Nº de ciudades: {df_items['city'].nunique()}")

Nº de ciudades: 11569


Cuando normalizamos vemos que el número de ciudades disminuye. Aun así, faltarían muchos más métodos que aplicar para llegar a normalizar el campo.

**NOTA** Por otro lado, mientras que estabamos limpiando datos, nos dimos cuenta que hay valores númericos para ciudades y códigos postales imputados como nombres.

In [87]:
df_city = df_items[(df_items['city'].notna()) & (df_items['city'].str.isnumeric())]
print(f"Nº ciudad con dígitos: {df_city.shape[0]}")
df_city[['num_order','city','zipcode']].sample(5,random_state=seed)

Nº ciudad con dígitos: 802


,num_order,city,zipcode
140206,2000185acbdce1eb1ff992920ba55ab7,08014,08014
794517,be53ac9ff585d3bb19950d1a5dd27de3,1500,409
613185,cd2574eba675f621839adb548d382828,28027,Madrid
888965,6c8481e2b51551d96013c8d243bde19a,10005,Cáceres
453354,c2c699bfd6eb8461a4a2ea5606b86e7b,08003,08003


In [88]:
df_zip = df_items[(df_items['zipcode'].notna()) & (df_items['zipcode'].notna())]
df_zip = df_zip[(df_zip['city'].str.isnumeric()) & (~df_zip['zipcode'].str.isnumeric())]
print(f"Nº códigos postales y ciudad invertidos: {df_zip.shape[0]}")
df_zip[['city','zipcode']].sample(10,random_state=seed)

Nº códigos postales y ciudad invertidos: 357


,city,zipcode
865154,28016,madrid
874602,36141,Vilaboa
758997,43800,Valls
567188,25600,Balaguer
469752,03660,NOVELDA
169158,46009,Valencia
917406,03206,Elche
804884,46182,LA CAÑADA
842221,13003,Ciudad Real
474900,08030,BARCELONA


### Duplicados

En apartados anteriores intuimos la presencia de registros duplicados, veamos a que se debe.

In [89]:
print(f"Regitros duplicados: {df_items.duplicated().sum()}")
df_items[df_items.duplicated()].head(5)

Regitros duplicados: 17599


,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
34,5992c1625cc4b93264594f98c0c88db3,8a4653d26eac38f05519dc529e5b4639,2017-01-01 08:31:58,24172,1,9.5160,12.14,8.0,528041acb3b5ddddc51f42c95a78c21a,murcia,30139
36,5992c1625cc4b93264594f98c0c88db3,9c7846d8386f7191de9640a4115dcb73,2017-01-01 08:31:58,6159,1,4.9140,7.40,8.0,528041acb3b5ddddc51f42c95a78c21a,murcia,30139
37,5992c1625cc4b93264594f98c0c88db3,80a5e455e90a2c8753662b71abed5d10,2017-01-01 08:31:58,10315,1,4.0788,5.58,8.0,528041acb3b5ddddc51f42c95a78c21a,murcia,30139
39,5992c1625cc4b93264594f98c0c88db3,8f29bf49f01d7e6b2d516f8bd6cfe435,2017-01-01 08:31:58,9048,1,2.3900,2.60,8.0,528041acb3b5ddddc51f42c95a78c21a,murcia,30139
40,5992c1625cc4b93264594f98c0c88db3,257d5601d124014aa1d583af2a070432,2017-01-01 08:31:58,30489,1,1.2700,4.81,8.0,528041acb3b5ddddc51f42c95a78c21a,murcia,30139


### Clientes que más compran

Clientes que más pedidos hacen

In [90]:
df_clients = df_items.groupby(['customer_id', 'num_order'],as_index=False).agg({'qty_ordered':'sum'}) # num_order se repite, por tanto no se puede agregar por el de primeras
df_clients = df_clients.groupby(['customer_id'],as_index=False).agg({'num_order':'count', 'qty_ordered':'sum'})
df_clients = df_clients.sort_values('num_order',ascending=False)
df_clients.head()

,customer_id,num_order,qty_ordered
83564,b46e4d83eb0c080e114a32ee1f842b7d,104,550
107769,e86e234e3b71bfba9a0ab95ab0de36b1,72,463
69102,94c74109038dc85def6ac232828b2131,72,413
118240,fec46170b4980fd6432797ed6ecdf7c1,64,156
106391,e597a4ec127e0dd5e19533b68edbc92f,60,673


Clientes que han comprado una mayor cantidad de productos

In [91]:
df_clientes = df_clients.sort_values('qty_ordered',ascending=False)
df_clientes.head()

,customer_id,num_order,qty_ordered
45115,60d097f001dd2ee8c9989434307a5c2b,27,1064
29375,3eda45623e6304ce8966c928a6aa7084,26,801
106391,e597a4ec127e0dd5e19533b68edbc92f,60,673
79725,ac02166ad1e62e310dc1a89e2ec24ce7,38,622
81154,af21079e8d355aded105bd4ce03034e0,6,600


### Ventas totales

En este apartado veremos las fechas en las cuáles se producen más ventas

In [92]:
# Modificamos la columna para que solo muestre la fecha sin la hora
df_items.loc[:, 'date'] = df_items['created_at'].dt.date
df_items[['created_at','date']].sample(5,random_state=seed)

,created_at,date
117430,2017-06-06 19:47:26,2017-06-06
835866,2018-11-24 14:30:23,2018-11-24
298248,2017-11-27 09:23:45,2017-11-27
103857,2017-05-13 08:27:06,2017-05-13
180683,2017-07-29 18:32:54,2017-07-29


Agrupamos por fecha para saber la cantidad de productos y el número de pedidos realizados por cada día

In [93]:
df_temp = df_items.groupby(['date', 'num_order']).agg({'qty_ordered':'sum'}).reset_index() # num_order se repite, por tanto no se puede agregar por el de primeras
df_temp = df_temp.groupby(['date']).agg({'num_order':'count', 'qty_ordered':'sum'}).reset_index()
df_temp.sample(5,random_state=seed)

,date,num_order,qty_ordered
685,2018-11-18,1524,8578
115,2017-04-26,423,2485
220,2017-08-09,432,1949
420,2018-02-26,152,465
378,2018-01-15,538,3317


In [94]:
fig = px.area(df_temp, x="date", y="num_order", title='Pedidos realizados por día',)
fig.show()

Vemos los picos de pedidos en los meses como Noviembre (BlackFriday) y picos en las rebajas tanto de invierno como de verano

In [95]:
fig = px.area(df_temp, x="date", y="qty_ordered", title='Cantidad de productos pedidos por día')
fig.show()

La cantidad de productos pedidos por día es muy similar en forma a la anterior gráfica, por lo que tienen cierta correlación. Aun así, nos debemos fijar en la escala del eje Y.

In [96]:
df_temp.corr()

,num_order,qty_ordered
num_order,1.000000,0.969879
qty_ordered,0.969879,1.000000


### Ventas por localización

Puesto que hemos sacado las coordenadas de cada localización podemos mostrar visualmente donde se han producido más ventas

In [97]:
df_temp = df_items.groupby(['city', 'num_order'],as_index=False).agg({'qty_ordered':'sum'}) # num_order se repite, por tanto no se puede agregar por el de primeras
df_temp = df_temp.groupby(['city'],as_index=False).agg({'num_order':'count', 'qty_ordered':'sum'})
df_temp.sort_values('num_order',ascending=False).head(10)

,city,num_order,qty_ordered
6050,madrid,30982,142546
1403,barcelona,12936,61839
10574,valencia,6488,28350
9631,sevilla,4358,19099
1794,bilbao,2740,12890
10620,valladolid,2156,9731
683,alicante,1929,8889
11469,zaragoza,1897,7905
346,albacete,1889,7962
6140,malaga,1802,7674


Dado que no hemos limpiado aun los datos, los pedidos por localización no son correctos, tal y como podemos ver en la muestra. Algunas ciudades como Madrid se repiten varias veces porque están escritas de forma distinta. Aquí es donde hay que remarcar la importancia de trabajar los datos.

## Conclusiones

A partir de este análisis hemos deducido:

* Una misma localización puede estar representada de muchas diferentes maneras.
* Los datos que faltan son pocos. De los campos *base_cost*, *city* y *zipcode* que faltan podemos guiarnos de otras ventas para intentar recuperarlos.
* Hay productos donde el precio base es negativo.
* Existen productos donde el precio de venta es inferior al precio base.
* Todos los pedidos tienen aplicado algún tipo de descuento.
* Existe un pico de ventas en días especiales, sobre todo black friday.